In [1]:
import pickle as pkl
import numpy as np
import pandas as pd
import sklearn
from twitter_nlp_toolkit.tweet_sentiment_classifier import tweet_sentiment_classifier
from twitter_nlp_toolkit.file_fetcher import file_fetcher
from zipfile import ZipFile

Using TensorFlow backend.


In [2]:
chunk = 1 # Fraction of data to train on - you can reduce for debugging for speed
redownload=True
model_path = 'Models'


Here we download pre-trained models and a validation dataset. The models have been trained on the Sentiment140 dataset, taken form here: https://www.kaggle.com/kazanova/sentiment140

The validation data is hand-labeled airline customer feedback taken from https://www.figure-eight.com/data-for-everyone/

In [3]:
if redownload: 
    
    # Validation data
    file_fetcher.download_file('https://www.dropbox.com/s/440m6x07bjg6c0h/Tweets.zipdl=1',"Tweets.zip")
    
    # Compressed model
    # Note that this model has only been trained on 5% of the training data
    # Update when better-trained model is available
    file_fetcher.download_file("https://www.dropbox.com/s/i88eqlja56xncyx/model_test_05.zip?dl=1","Models.zip")
    
    # Extract all the contents of zip file in current directory
    with ZipFile('Models.zip', 'r') as zipObj:
        zipObj.extractall(path=model_path)

Tweets.zip: 298kB [00:00, 1.79MB/s]
Models.zip: 100%|##########| 1.82G/1.82G [02:00<00:00, 16.2MB/s]


In [4]:
# Load the validation data

test_data = pd.read_csv('Tweets.zip', header=0, names=['Index', 'Sentiment', 'Sentiment_confidence',
                                                                'Negative_reason', 'Negative_reason_confidence',
                                                                'Airline', 'Airline_sentiment_gold', 'Handle',
                                                                'Negative_reason_gold', 'Retweet_count', 'Text',
                                                                'Tweet_coord', 'Time', 'Location', 'Timezone'])

In [5]:
# Remove the unlabeled test data

test_data['Labels'] = (test_data['Sentiment'] == 'positive') * 2
test_data['Labels'] = test_data['Labels'] + (test_data['Sentiment'] == 'neutral') * 1
test_data['Labels'] = test_data['Labels'] / 2

test_data.set_index('Labels')
test_data = test_data[test_data.Labels != 0.5]

In [6]:
# Executing this cell takes about 30s on a laptop

Classifier = tweet_sentiment_classifier.SentimentAnalyzer()
Classifier.load_models(path=model_path)

BoW model Models\bow loaded successfully
BoW model Models\bow_005_1 loaded successfully
BoW model Models\bow_005_2 loaded successfully
BoW model Models\bow_005_3 loaded successfully
Pre-trained embedding model loaded successfully
Pre-trained embedding model loaded successfully
Pre-trained embedding model loaded successfully
Pre-trained embedding model loaded successfully
LSTM model Models\lstm loaded successfully
LSTM model Models\lstm_005_1 loaded successfully
LSTM model Models\lstm_005_2 loaded successfully
LSTM model Models\lstm_005_3 loaded successfully
BoW model Models\model_test_05 loaded successfully


We santiy check the models: 

In [7]:
Classifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])


array([1., 0., 1., 0.])

We test the model on an airline customer feedback dataset.

In [8]:
# Executing this cell takes several minuites on a laptop

predictions = Classifier.predict(test_data['Text'])


In [9]:
print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(test_data['Labels'], predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(test_data['Labels'], predictions)))
sklearn.metrics.confusion_matrix(test_data['Labels'], predictions)

Test Accuracy:  0.773
Test MCC:  0.528


array([[6806, 2372],
       [ 246, 2117]], dtype=int64)

We have accuracy of just under 80%.


We split our evaluation dataset into validation and testing and eliminate the worst-performing models

In [10]:
valX, testX, valY, testY = sklearn.model_selection.train_test_split(test_data['Text'], test_data['Labels'], test_size=0.5, stratify=test_data['Labels'])

In [11]:
# Executing this cell takes several minuites on a laptop


Classifier.trim_models(valX, valY, threshold=0.7)

Model Models\bow score: 0.801
Model Models\bow_005_1 score: 0.738
Model Models\bow_005_2 score: 0.756
Model Models\bow_005_3 score: 0.746
Model Models\glove score: 0.765
Model Models\glove_005_1 score: 0.724
Model Models\glove_005_2 score: 0.677
Deleting model Models\glove_005_2
Model Models\glove_005_3 score: 0.673
Deleting model Models\glove_005_3
Model Models\lstm score: 0.689
Deleting model Models\lstm
Model Models\lstm_005_1 score: 0.652
Deleting model Models\lstm_005_1
Model Models\lstm_005_2 score: 0.631
Deleting model Models\lstm_005_2
Model Models\lstm_005_3 score: 0.648
Deleting model Models\lstm_005_3
Model Models\model_test_05 score: 0.801


Our custom-embedding models performed poorly on this dataset and have been pruned. 

In [12]:
predictions = Classifier.predict(testX)

print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(testY, predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(testY, predictions)))
sklearn.metrics.confusion_matrix(testY, predictions)

Test Accuracy:  0.784
Test MCC:  0.535


array([[3485, 1104],
       [ 140, 1042]], dtype=int64)

Pruning our models had a minor impact on our classification performance, bringing us to an acceptible ~77%. 

To improve our accuracy, we can refine the models on our airline data. The early stopping procedure (enabled by default to use 20% of the training data for validation) should minimize overfitting.

In [13]:
Classifier.refine(valX, valY)
test_predictions = Classifier.predict(testX)

predictions = Classifier.predict(testX)

Train on 4616 samples, validate on 1154 samples
Epoch 1/500
4616/4616 [==============================] - 2s 380us/step - loss: 0.4262 - acc: 0.8234 - val_loss: 0.3077 - val_acc: 0.8839
Epoch 2/500
4616/4616 [==============================] - 1s 275us/step - loss: 0.3701 - acc: 0.8533 - val_loss: 0.2694 - val_acc: 0.8934
Epoch 3/500
4616/4616 [==============================] - 1s 274us/step - loss: 0.3255 - acc: 0.8726 - val_loss: 0.2635 - val_acc: 0.9073
Epoch 4/500
4616/4616 [==============================] - 1s 280us/step - loss: 0.3079 - acc: 0.8865 - val_loss: 0.2657 - val_acc: 0.9090
Epoch 5/500
4616/4616 [==============================] - 1s 276us/step - loss: 0.2964 - acc: 0.8925 - val_loss: 0.2816 - val_acc: 0.9090
Epoch 6/500
4616/4616 [==============================] - 1s 278us/step - loss: 0.3146 - acc: 0.8858 - val_loss: 0.3094 - val_acc: 0.9073
Epoch 7/500
4616/4616 [==============================] - 1s 277us/step - loss: 0.3400 - acc: 0.8884 - val_loss: 0.3016 - val_acc: 

Epoch 60/500
4616/4616 [==============================] - 1s 289us/step - loss: 0.2058 - acc: 0.9168 - val_loss: 0.1993 - val_acc: 0.9333
Epoch 61/500
4616/4616 [==============================] - 1s 281us/step - loss: 0.2069 - acc: 0.9153 - val_loss: 0.1987 - val_acc: 0.9315
Epoch 62/500
4616/4616 [==============================] - 1s 284us/step - loss: 0.2174 - acc: 0.9144 - val_loss: 0.1981 - val_acc: 0.9315
Epoch 63/500
4616/4616 [==============================] - 1s 282us/step - loss: 0.2217 - acc: 0.9146 - val_loss: 0.1977 - val_acc: 0.9315
Epoch 64/500
4616/4616 [==============================] - 1s 278us/step - loss: 0.2126 - acc: 0.9157 - val_loss: 0.1975 - val_acc: 0.9315
Epoch 65/500
4616/4616 [==============================] - 1s 277us/step - loss: 0.2047 - acc: 0.9133 - val_loss: 0.1974 - val_acc: 0.9324
Epoch 66/500
4616/4616 [==============================] - 1s 273us/step - loss: 0.2142 - acc: 0.9170 - val_loss: 0.1972 - val_acc: 0.9324
Epoch 67/500
4616/4616 [==========

4616/4616 [==============================] - 1s 275us/step - loss: 0.1764 - acc: 0.9218 - val_loss: 0.2021 - val_acc: 0.9324
Epoch 120/500
4616/4616 [==============================] - 1s 272us/step - loss: 0.1817 - acc: 0.9222 - val_loss: 0.2020 - val_acc: 0.9324
Epoch 121/500
4616/4616 [==============================] - 1s 272us/step - loss: 0.1759 - acc: 0.9216 - val_loss: 0.2019 - val_acc: 0.9315
Epoch 122/500
4616/4616 [==============================] - 1s 274us/step - loss: 0.1819 - acc: 0.9261 - val_loss: 0.2018 - val_acc: 0.9324
Epoch 123/500
4616/4616 [==============================] - 1s 273us/step - loss: 0.1785 - acc: 0.9244 - val_loss: 0.2017 - val_acc: 0.9333
Epoch 124/500
4616/4616 [==============================] - 1s 273us/step - loss: 0.1882 - acc: 0.9255 - val_loss: 0.2016 - val_acc: 0.9359
Epoch 125/500
4616/4616 [==============================] - 1s 276us/step - loss: 0.1708 - acc: 0.9268 - val_loss: 0.2015 - val_acc: 0.9359
Epoch 126/500
4616/4616 [================

Epoch 19/500
4616/4616 [==============================] - 1s 276us/step - loss: 0.2719 - acc: 0.8958 - val_loss: 0.2394 - val_acc: 0.9168
Epoch 20/500
4616/4616 [==============================] - 1s 274us/step - loss: 0.2880 - acc: 0.8964 - val_loss: 0.2390 - val_acc: 0.9177
Epoch 21/500
4616/4616 [==============================] - 1s 277us/step - loss: 0.2630 - acc: 0.8980 - val_loss: 0.2578 - val_acc: 0.9185
Epoch 22/500
4616/4616 [==============================] - 1s 277us/step - loss: 0.2876 - acc: 0.8980 - val_loss: 0.2783 - val_acc: 0.9185
Epoch 23/500
4616/4616 [==============================] - 1s 274us/step - loss: 0.3093 - acc: 0.8995 - val_loss: 0.2859 - val_acc: 0.9194
Epoch 24/500
4616/4616 [==============================] - 1s 279us/step - loss: 0.2888 - acc: 0.9008 - val_loss: 0.2928 - val_acc: 0.9194
Epoch 25/500
4616/4616 [==============================] - 1s 280us/step - loss: 0.2745 - acc: 0.8995 - val_loss: 0.2714 - val_acc: 0.9194
Epoch 26/500
4616/4616 [==========

4616/4616 [==============================] - 1s 274us/step - loss: 0.2192 - acc: 0.9153 - val_loss: 0.2311 - val_acc: 0.9324
Epoch 79/500
4616/4616 [==============================] - 1s 276us/step - loss: 0.2167 - acc: 0.9133 - val_loss: 0.2314 - val_acc: 0.9324
Epoch 80/500
4616/4616 [==============================] - 1s 274us/step - loss: 0.2208 - acc: 0.9131 - val_loss: 0.2337 - val_acc: 0.9315
Epoch 81/500
4616/4616 [==============================] - 1s 277us/step - loss: 0.2221 - acc: 0.9142 - val_loss: 0.2513 - val_acc: 0.9307
Epoch 82/500
4616/4616 [==============================] - 1s 273us/step - loss: 0.2172 - acc: 0.9112 - val_loss: 0.2534 - val_acc: 0.9315
Epoch 83/500
4616/4616 [==============================] - 1s 274us/step - loss: 0.2260 - acc: 0.9146 - val_loss: 0.2476 - val_acc: 0.9324
Epoch 84/500
4616/4616 [==============================] - 1s 276us/step - loss: 0.2243 - acc: 0.9175 - val_loss: 0.2479 - val_acc: 0.9307
Epoch 85/500
4616/4616 [=======================

4616/4616 [==============================] - 1s 276us/step - loss: 0.1901 - acc: 0.9172 - val_loss: 0.2320 - val_acc: 0.9359
Epoch 138/500
4616/4616 [==============================] - 1s 276us/step - loss: 0.1876 - acc: 0.9220 - val_loss: 0.2256 - val_acc: 0.9359
Epoch 139/500
4616/4616 [==============================] - 1s 272us/step - loss: 0.1924 - acc: 0.9201 - val_loss: 0.2307 - val_acc: 0.9359
Epoch 140/500
4616/4616 [==============================] - 1s 275us/step - loss: 0.1892 - acc: 0.9209 - val_loss: 0.2225 - val_acc: 0.9359
Epoch 141/500
4616/4616 [==============================] - 1s 274us/step - loss: 0.1902 - acc: 0.9198 - val_loss: 0.2229 - val_acc: 0.9359
Epoch 142/500
4616/4616 [==============================] - 1s 277us/step - loss: 0.1876 - acc: 0.9218 - val_loss: 0.2303 - val_acc: 0.9359
Epoch 143/500
4616/4616 [==============================] - 1s 274us/step - loss: 0.1947 - acc: 0.9198 - val_loss: 0.2310 - val_acc: 0.9350
Epoch 144/500
4616/4616 [================

In [14]:
print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(testY, predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(testY, predictions)))
sklearn.metrics.confusion_matrix(testY, predictions)

Test Accuracy:  0.897
Test MCC:  0.656


array([[4511,   78],
       [ 519,  663]], dtype=int64)

Now we have accuracies of nearly 90%! 

In [15]:
Classifier.evaluate(testX, testY)

Model Models\bow score: 0.891
Model Models\bow_005_1 score: 0.891
Model Models\bow_005_2 score: 0.892
Model Models\bow_005_3 score: 0.889
Model Models\glove score: 0.883
Model Models\glove_005_1 score: 0.879
Model Models\model_test_05 score: 0.891


{'ensembled': 0.896551724137931,
 'Models\\bow': 0.8911800381216427,
 'Models\\bow_005_1': 0.8910067579275689,
 'Models\\bow_005_2': 0.8920464390920118,
 'Models\\bow_005_3': 0.8894472361809045,
 'Models\\glove': 0.8826893086120257,
 'Models\\glove_005_1': 0.8790504245364755,
 'Models\\model_test_05': 0.8911800381216427}